# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,13.10,81,32,13.97,NZ,1713755935
1,1,jamestown,42.0970,-79.2353,-1.01,85,3,2.13,US,1713755936
2,2,ribeira grande,38.5167,-28.7000,15.05,74,100,2.65,PT,1713755936
3,3,blackmans bay,-43.0167,147.3167,17.46,56,11,1.64,AU,1713755936
4,4,mount gambier,-37.8333,140.7667,22.35,23,2,3.35,AU,1713755936


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:

%%capture --no-display

# Configure the map plot
# Configure the map plot_3
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 620,
    frame_height = 500,
    size = "Humidity",
    scale = 1
    ,
    
)

# Display the map plot
map_plot




:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[(city_data_df['Max Temp'] < 30)]


# Drop any rows with null values
filtered_df.dropna(inplace=True)

# Display sample data
print(filtered_df)

     City_ID            City      Lat       Lng  Max Temp  Humidity  \
0          0        waitangi -43.9535 -176.5597     13.10        81   
1          1       jamestown  42.0970  -79.2353     -1.01        85   
2          2  ribeira grande  38.5167  -28.7000     15.05        74   
3          3   blackmans bay -43.0167  147.3167     17.46        56   
4          4   mount gambier -37.8333  140.7667     22.35        23   
..       ...             ...      ...       ...       ...       ...   
589      589   labrador city  52.9463  -66.9114     -3.90        75   
591      591         krupina  48.3554   19.0647      1.81       100   
592      592       tazacorte  28.6290  -17.9293     19.86        72   
593      593         tindouf  27.6711   -8.1474     14.32        76   
594      594       priozersk  61.0393   30.1291     -0.31        94   

     Cloudiness  Wind Speed Country        Date  
0            32       13.97      NZ  1713755935  
1             3        2.13      US  1713755936

C:\Users\jrobe\AppData\Local\Temp\ipykernel_13840\578257499.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.dropna(inplace=True)


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
print(hotel_df)

               City Country      Lat       Lng  Humidity Hotel Name
0          waitangi      NZ -43.9535 -176.5597        81           
1         jamestown      US  42.0970  -79.2353        85           
2    ribeira grande      PT  38.5167  -28.7000        74           
3     blackmans bay      AU -43.0167  147.3167        56           
4     mount gambier      AU -37.8333  140.7667        23           
..              ...     ...      ...       ...       ...        ...
590        bengkulu      ID  -3.8004  102.2655        68           
591         krupina      SK  48.3554   19.0647       100           
592       tazacorte      ES  28.6290  -17.9293        72           
593         tindouf      DZ  27.6711   -8.1474        76           
594       priozersk      RU  61.0393   30.1291        94           

[595 rows x 6 columns]


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
import requests

def find_nearest_hotel(Lat, Lng):
    # Geoapify API endpoint for finding hotels
    url = "https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect:-101.18657623062036,25.07930207869075,-101.06362376937768,24.978861381826217&limit=20&apiKey=geoapify_key"
    
    # Parameters for the API request
    params = {
        "Lat": Lat,
        "Lng": Lng,
        "categories": "lodging",
        "radius": 10000,  # 10,000 meters
        "apiKey": "geoapify_key"  
    }
    
    # Sending the API request
    response = requests.get(url, params=params)
    
    # Checking if the request was successful
    if response.status_code == 200:
        data = response.json()
        if 'features' in data and len(data['features']) > 0:
            # Retrieve information about the first hotel found
            hotel_info = data['features'][0]['properties']
            return hotel_info
    else:
        print("Error:", response.status_code)
        return None

# Iterate through each city in the filtered DataFrame
for index, row in filtered_df.iterrows():
    city_name = row['City']
    city_latitude = row['Lat']
    city_longitude = row['Lng']
    
    # Find the nearest hotel for the current city
    hotel_info = find_nearest_hotel(city_latitude, city_longitude)
    
    # Display the city and hotel information
    if hotel_info:
        print(f"For {city_name}, nearest hotel is {hotel_info['name']}, located at {hotel_info['address']}.")
    else:
        print(f"No hotels found near {city_name}.")



Error: 401
No hotels found near waitangi.
Error: 401
No hotels found near jamestown.
Error: 401
No hotels found near ribeira grande.
Error: 401
No hotels found near blackmans bay.
Error: 401
No hotels found near mount gambier.
Error: 401
No hotels found near chernyshevsk.
Error: 401
No hotels found near edinburgh of the seven seas.


In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "radius": radius,
    "limit": 1  # Limiting to retrieve only the first hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point:{latitude},{longitude}"
    params["bias"] = f"point:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://maps.geoapify.com/v1/tile/carto/{z}/{x}/{y}.png?&apiKey="


    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

Starting hotel search
waitangi - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
mount gambier - nearest hotel: No hotel found
bazpur - nearest hotel: No hotel found
chernyshevsk - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
selfoss - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
zhosaly - nearest hotel: No hotel found
egilsstadir - nearest hotel: No hotel found
kaya - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
aksu - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
castillejos - nearest hotel: No hotel found
kill devil hills - nearest 

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)